This notebook provides an example of taking a focus sequence on a particular target.  It includes every step, from slewing to the target to adjusting the focus position.  
Displaying the focus fit is handled by a separate ipython notebook, MeasureFocus.ipynb, so that the two can be run side by side. 
Written: 02/18/2020 by Sasha Brownsberger
Modified: 
02/19/2020 by Sasha Brownsberger

In [5]:
import sys
import math
import os
import asyncio
import logging
import time

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [3]:
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/sashabrownsberger/ospl.xml"

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)

In [ ]:
await asyncio.gather(attcs.start_task, latiss.start_task)

##You must run the below lines only if you have not yet enabled the system (if you have taken biases, flats, etc, this should already be done)##

(There is no harm in running it repeatedly, however) 

In [2]:
#And enable the system:
await attcs.enable()
await latiss.enable()

In [3]:
##End enable scripts.##  

This is a generally useful function for cleanly rounding numbers for display purposes.

Slew to to the field of interest.  

Note that by having the slewing go to a particular RA/Dec position, we can specify the target_name as being a focus.  

In [7]:
#Focus units are given in mm 
focus_center = -0.2155
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str([pos for pos in focus_positions]))

focus_positions = [-0.2955, -0.27549999999999997, -0.2555, -0.2355, -0.2155, -0.1955, -0.1755, -0.1555, -0.1355]


User should specify the observing parameters that they want, including: 
filter, grating, exposure time, ...

In [ ]:
exp_time = 5.0 
filter_str = 'empty_1'
grating_str = 'ronchi90lpmm'

And now we can loop through the specified focus values: 

In [6]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    focus_image = await latiss.take_engtest(exptime=exp_time, n=1, filter=filter_str, grating=grating_str) 
    print ('Newest image id is = ' + str(focus_image ) )
    focus_images[i] = int(focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)
 

NameError: name 'focus_positions' is not defined